# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe")
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    print('網頁更新中…')
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    
    last_height = new_height

網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…
網頁更新中…


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 新店幸運得主現身！　七年級生靠女友「幸運密碼」刮中100萬

▲「2000萬超級紅包」百萬得主現身，分享幸運中獎過程。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
台彩近期推出「2000萬超級紅包」，今（18）日三獎百萬得主現身新北市新店區的吉優彩券行，這名住在新店的七年級生用「幸運密碼」，當天刮了5張都沒中，直到第6張以他女友的出生年月挑選刮刮樂，幸運中了百萬獎金，不但比年終獎金高很多，也比一年的薪水還要多一點，直呼「聽到當下很開心」，希望下次還有機會中大獎。
農曆春節即將到來，台彩「2000萬超級紅包」就已開出三獎100萬元，幸運得主是一位來自新北市新店區、年約30歲的七年級生，他今天現身吉優彩券行接受媒體訪問時表示，過去都有買彩券、刮刮樂的習慣，平均3天就會買一次，有時下班路過就會買個一、兩張。
這名七年級生透露，他自己固定有5、6組的「幸運密碼」，而所謂的幸運密碼是指出生年月、身分證字號，且以家人、朋友的號碼來挑選，而當天其實他刮了5張都尚未中獎，直到第6張改以女友的出生年月就幸運中了百萬元，打破過去買一張500元刮刮樂，幸運中5000元的紀錄。
▲吉優彩券行業者透露，曾在2017年開出今彩539 頭獎，這次則刮出「2000萬超級紅包」的三獎。（圖／記者紀佳妘攝）
他也提及，刮中獎金的當下，以為自己中10萬元，拿給彩券行阿姨才發現是100萬元，直呼「當下聽到很開心」，同時他也很低調，沒有讓家人知道，也沒讓女友知道幸運中獎的事情，今天還是找藉口從家裡出來彩券行受訪，希望讓大家知道彩券還是會中獎，另一方面也能幫助弱勢團體做公益。
至於這筆意外的獎金會如何運用，他說，這筆百萬獎金確實比年終獎金還高很多，也比一年得薪水還要多一點，因此除了用來旅遊外，其餘的錢則會用來儲蓄。
另名路過彩券行的阮先生看到彩券行擠滿了人潮，特別花了3.6萬元買一本「2000萬超級紅包」來試試手氣。他表示，小孩剛出生6天，這次也想沾沾喜氣，期望也能一圓發財夢，也沒有特別挑選號碼，有信心「刮什麼都會中獎」。
----------------------------------------------------------------------

以台商在意的缺地問題為例，官員表示，經盤點截至2022年為止，全台可提供設廠的產業用地約1,827公頃，其中，立即可供設廠用地則有716公頃，多數集中於中南部，政府將採取「只租不售」方式逐步釋出。
對於土地取得較不易的北部地區，政府則祭出容積獎勵，推動工業區更新立體化方案，官員指出，截至2019年11月底為止，全台已有30家廠商提出申請，可望創造224.81億元新增投資、提供7.17公頃產業空間。
在中美貿易戰的大國角力之下，市場多數對全球經濟前景悲觀，但沈榮津反而認為是台商全球化佈局的轉骨契機，依他推論，在中美貿易戰的推波助瀾下，製造業未來將不會集中在世界工廠中國大陸，而將採取區塊、分散式的生產，如何讓生產基地更貼近市場需求，將是台商決定全球投資佈局的關鍵。
----------------------------------------------------------------------

[10] 小英勝選財經篇／Google、臉書、微軟擴大在台投資　國發會分析外資看好台灣2大原因！

▲外界分析，蔡英文連任，可望吸引更多外資投資台灣。（圖／路透）
記者邱倢芯／台北報導
2020總統大選落幕，現任總統蔡英文高票當選連任，外界分析，這可望吸引更多外資投資台灣，而早已在台扎根的外企，如Google、Facebook、微軟、Amazon，乃至於科技龍頭Apple，都可望加重在台投資力道。
且在2019年3月時，Google還進一步宣布將在新北遠東通訊園區當中設立新辦公室，並擴大在台徵才計畫；當時Google裝置與服務資深副總裁Rick Osterloh還特地來台宣布此一好消息。
據了解，目前Google除了在101大樓中設有700~800人的辦公室，以及預定在2020年底正式開幕的遠東通訊園區辦公室外，在台灣還有從宏達電買下的手機部門（新店），在彰化設置資料中心、台南綠能電廠，以及在台南另一座仍在建置階段的資料中心，展現Google在台灣投資信心。
▼2019年3月時，Google進一步宣布將在新北遠東通訊園區當中設立新辦公室，並擴大在台徵才計畫。（圖／記者邱倢芯攝）
蔡尋求兩岸關係穩定　提高外資投資意願
國發會指出，這次大選前，部分外資對於台灣的投資意願仍屬游移，因為總統大選的不確定性將會影響投資決策；而現在選舉結果已出爐，蔡英文連任成功，將有助外資投資。

▲▼美國。（圖／pixabay）
記者吳靜君／台北報導
美國國稅局(IRS)近期公布「特定前公民適用之救濟程序」，針對符合特定條件的已棄籍或計畫棄籍的美國公民提供一個合規化的機會。
資誠稅務諮詢顧問執行董事蘇宥人表示，當美國公民因各種原因，包含但不限於生活事業重心調整、家庭因素等，而想要放棄美國籍，除了「移民法律上的棄籍」，也需要向美國國稅局進行棄籍申報，也就是「稅務上的棄籍」。實務上，部分棄籍者誤以為滿足「移民法律上的棄籍」條件與程序後便完成棄籍，卻忽略「稅務上的棄籍」相關義務。
根據美國現行稅法規定，棄籍者於棄籍時的全球資產市價淨值達200萬美元者，或過去五年每年平均應納稅額超過棄籍當年的法定門檻，2020年為17萬1,000美元，或在棄籍前五個年度未完全遵守所有美國聯邦納稅義務者，即視為「covered expatriate」，此身分的主要影響是，須視同於棄籍前一日以市價處分全球所有資產並據以計算「棄籍稅」，於棄籍當年度申報完稅。
此外，無論財產及應納稅額達到門檻與否，所有棄籍者皆須向IRS申報Form 8854，才能為自身的美國申報及納稅義務畫下句點。實務上，部分棄籍者雖然應納稅額及財產市值均遠低於「covered expatriate」的門檻，但是因為在棄籍時忘記申報Form 8854，或者要申報、補申報Form 8854時發現之前美國稅遵循有缺失，甚至完全沒申報過美國稅，因此無法在Form 8854聲明在前五個年度均遵守所有美國聯邦納稅義務。
根據美國稅法的規定此種情況均會落入「covered expatriate」的範圍，但若想要補申報，則除了補繳稅款以外也會產生各種利息、滯納金、罰鍰等，影響到棄籍者的合規化意願，因而增加美國稅務風險。
放棄美國籍者　有不落入補繳棄籍稅的機會
蘇宥人指出，美國國稅局(IRS)公布的「特定前公民適用的救濟程序」，提供給符合特定條件的已棄籍或計畫棄籍的美國公民一個合規化的機會。
符合資格條件的已棄籍或計劃棄籍的美國公民，但是新救濟程序，可透過此程序而合規化並完成稅法上的棄籍程序，除了不落入「棄籍稅」範圍外，也不需要補繳稅額及相關利息及罰鍰。
▲放棄美籍要打行政救濟，必須符合六大程序與條件。（圖／路透）
根據IRS公布的說明，若棄籍者均沒有達到以上所說明的應納稅額及財產門檻，而且在棄籍前五個年度確實有遵守所有美國聯邦納稅義務

記者張方瑀／綜合報導
白宮貿易顧問納瓦洛（Peter Navarro）接受媒體訪問時指出，本周三（15日）、周四（16日）是美國貿易政策歷史上最偉大的兩天；因為這兩天不但簽署了中美第一階段貿易協議，還談妥了「美國-墨西哥-加拿大協定」（USMCA）。納瓦洛也強調，若是中國不履行協議內容，將會採取應對的報復措施。
納瓦洛接受《福斯財經新聞》訪問時表示，「能夠談妥一個協議其實就很足夠了，但我們在兩天內簽署了兩個協議。」納瓦洛說，美國和加拿大、墨西哥、中國簽了協議，同時也在和日本、韓國協商中，「我只能說，哇！」納瓦洛也指，目前美國市場對於這些協議的反應非常好，因為這意味著美國人民有更多就業機會。
不過，納瓦洛仍擔心中國不會履行協議內容。他表示，美方為此計畫了90天的裁決期限，「若是貿易代表萊特海澤（Robert Lighthizer）接到申訴的話，必須在90天內做出裁決。」納瓦洛強調，如果美國不滿意中國做法的話，可以執行報復行動，且中方不可對此採取應對措施。
----------------------------------------------------------------------

[31] 財訊快報／Mini LED高階市場點亮了

談了數年的Mini LED，供應鏈技術出現突破，高階中尺寸產品線成本甜蜜點浮現，2020年量產，成為LED產業突破陸廠價格圍剿下的利器，未來3～5年背光滲透率可望逐步提升。
文／巫彩蓮
蘋果，向來是科技產品高規格的推手，iPhone一路從視網膜顯示器（Retina Display）、AMOLED面板，總能創造話題性，並為下世代規格定錨，2020年顯示器產品焦點莫過次毫米發光二極體（Mini LED）背光採用，這也將為LED產業注入一股新氣象。
蘋果率先在ProDisplay採用類Mini LED背光解決方案，係由576顆藍色LED組成的超亮陣列，對比度來到100萬：1，亮度達到1600尼特；接著新款16吋MacBook Pro亮相，該款產品解析度3072×1920，最大亮度到500尼特，外傳背光係由晶電供應Mini LED晶粒，搭配韓廠樂金顯示器（LGD）面板。
Mini LED背光抗衡OLED
LED業界寄予厚望Mini LED背光，在蘋果顯示器（Monitor）、NB（筆記型電腦）等高階顯示器量產有了眉目，不再是紙上談

Ａ：前陣子中美貿易戰，上證指數一度跌到二千四百點，除了國內資金進場護盤外，不少國際資金也悄悄布局，而今，停戰合約簽訂，可望帶動陸股衝出一波補漲行情，就長線來看，只要國際沒有大利空消息，陸股將會在今年一路衝出，想要放長線或分散投資標的的朋友，不妨可以考慮。
投資陸股需在大陸開立證券帳戶，但近年中國金融法規越修越嚴，一般投資大眾很難直接買入大陸股票，所以在台灣較常見投資陸股方式是買入A50海外期貨或是陸股相關的ETF。
A50於新加坡交易所掛牌，隸屬於海外期貨範圍，內容包含了在上海或深圳證交所上市的股票中市值最大的五十家Ａ股公司，能夠直接反映中國權值Ａ股的價格，因此，對陸股不熟悉但想賺取其漲幅價差的投資朋友，是一個簡單的投資方式，值得注意的是，期貨有結算問題，如果想抱長線需要注意轉倉。
而投資陸股ETF相對來說簡單很多，只要到各大證券公司開戶，就可以買入陸股相關ETF，由於ETF連動的是上證或深圳指數，也省去挑股的麻煩，只要確認長線是多頭，可以利用定期定額方式買入，擺放幾年，賺取陸股成長空間，適合新手或沒時間研究的投資朋友們參與。
就技術分析來看，醞釀七個多月的盤整發動，有機會向上衝一波挑戰高點三二八八，加上國際資金到位，補漲行情蓄勢待發，想尋找新投資標的朋友不妨趁機先卡位，等待國際法人為你抬轎。
----------------------------------------------------------------------

[37] 理財周刊／選後行情續燒　嚴防類股快速輪動

文／理周教育學苑．方鈞
Ｑ：台股在創了12,197高點後，連跌四天，總計400多點，跌破萬二大關，嚇壞投資朋友，以為台股就此完蛋，好在國際市場美股止跌，外資偕同內資加碼買進，穩住這場恐慌，越到高點，越是如履薄冰，如今，選舉已經結束，台股後勢是否依然可期？
Ａ：觀察一月十三日選後外資籌碼流向，未平倉口數增加一一八三口，呈現小幅增長，加權指數早盤在金融股表態慶祝下，終場以一二一一三點作收，成交量一四二六億元，拉尾盤態勢明顯，依外資及尾盤拉紅狀態，台股多頭格局尚未結束，未來有機會續攻挑戰高點。
近日外資買超前幾大股票不再以權值股占主要大宗，跌深的電子股、5G概念股或是金融股成為外資最愛，權值股領頭是加權指數能夠創高關鍵，但創高後維持不跌則需各類股輪動，等所有資金到位後才會有新一波的行情。

▲亞馬遜執行長貝索斯（Jeff Bezos）。（圖／路透社）
記者謝仁傑／綜合報導
據外媒報導，在亞馬遜執行長貝佐斯（Jeff Bezos）訪問印度期間，印度貿易部長戈亞爾（Piyush Goyal）表示，亞馬遜投資的10億美元（約新台幣299.17億元）並沒有給印度帶來什麼好處。這使得亞馬遜與印度之間的緊張關係更加白熱化。
貝佐斯周三表示，亞馬遜將投資10億美元，在該國實現小企業上網。但亞馬遜正與沃爾瑪旗下的Flipkart，正面臨著印度越來越多實體零售商的批評，他們指責這些美國巨頭斥資數十億美元來提供大幅折扣、歧視小型零售商、違反了印度法律。
美國的大公司們對這些指控均加以否認，亞馬遜對於印度貿易部長的這一表態也並未發表任何說法。
在貝佐斯訪問印度之前，印度競爭委員會已對亞馬遜和Flipkart展開了調查，以瞭解有關大幅折扣的指控以及這些公司是否歧視小型零售商。戈亞爾說：「這是每個印度人都關注的問題。」
據先前《經濟時報》報導，截至2019年3月，亞馬遜在印度的市場部門報告虧損568.5億盧比（約新台幣239.87億元）。消息人士向路透社透露，儘管亞馬遜一再提出要求會面印度總理莫迪的要求，但在這段時間訪問印度的時間裡應該沒什麼機會。
----------------------------------------------------------------------

[47] Gucci四月進駐遠東SOGO百貨　黃晴雯：精品五大天王齊聚挹注營運可期

▲遠東SOGO百貨董事長黃晴雯(圖中)表示，精品品牌GUCCI預計於四月進駐復興館，屆時法國名牌五大天王齊聚，挹注營運可期。（圖／記者林淑慧攝）
記者林淑慧／台北報導
邁入秋冬消費旺季，零售百貨業擘劃來年展望。遠東SOGO百貨董事長黃晴雯表示，SOGO透過六大社群經營分眾客群有成，精品去年成長性佳，並宣佈GUCCI預計於四月進駐復興館，屆時精品名牌五大天王齊聚，挹注營運可期。
遠東SOGO百貨去年周年慶檔期，精品業成長表現亮眼，推升復興館營收表現躍進，「台灣能聚集法國四大天王的就是復興館，今年四月還要再迎來GUCCI，屆時全球精品產值最高的五大品牌，將全部齊聚到復興館。」黃晴雯說。
為了力抗電商吸客，遠東SOGO百貨內部成立六大社群，依據顧客消費力進行分眾經營，從頂級會員的VVIP Club、主力顧客的VI

▲根據金管會統計，過去5年各家銀行裁撤分行總家數為80家、裁撤總家數則為36家。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
金管會最新統計指出，2015年至2019年國銀裁撤總分行數為80家，同時期核准新設總分行數則為36家，顯示各銀行新設分行趕不上裁撤分行的速度；此外，凱基銀、京城銀分別在2017年向金管會遞件申請新設3家分行，不過皆在2019年放棄不開業。
根據金管會銀行局統計，截至2019年11月30日，本國銀行分支機構家數共有3,404家，國內每10萬成年人平均擁有本國銀行分行家數為17家。
觀察近5年來國銀分支機構表現，2015年核准新設4家分行、裁撤23家分行，2016年增設4家、裁撤22家分行，2017年增設9家分行、裁撤14家分行，2018年增設10家分行、裁撤15家分行，2019年增設9家分行、裁撤6家分行。
從數據來看，過去5年各家銀行裁撤分行總家數為80家、裁撤總家數則為36家，由此可見，銀行裁撤總分行數比金管會核准新設分行足足多了2.2倍，換句話說，各銀行新設分行趕不上裁撤分行的速度。
金管會銀行局表示，目前金管會已核准凱基銀行、陽信銀行、京城銀行可設立4家分行，其中凱基銀行在2017年申請新設2家分行，不過由於找不到適合的地點、因應數位化等問題，自行評估效益，後在2019年向金管會表達決定不開業。
至於京城銀行也是在2017年核准新設分行，不過到了2019年也遲遲尚未有動作，銀行局指出，按照規定，金管會核准新設分行後，1年內必須要開業，超過1年的話就要申請延長，若沒有申請延長就視同「放棄」。
此外，陽信銀行也有被核准新設1家分行，銀行局表示，目前該銀行有申請延長至2020年12月；外界關注今年是否已有銀行申請裁撤分行，銀行局則說，目前尚未收到申請案，因此尚未有銀行要裁撤分行。
----------------------------------------------------------------------

[57] 強化ESG風險管理並引導重視氣候風險　證交所修訂企業社會責任報告書編制辦法

▲ 台灣證券交易所。（圖／記者李瑞瑾攝）
記者李瑞瑾／台北報導
臺灣證券交易所表示，參考國際非財務資訊揭露趨勢，已於109年1月2日公告修訂「上市公司編製與申報企業社會責任報告書作業辦法」，以提升上市公司非財務資訊揭露品質並落實企業永續發展

▲晶華酒店集團董事長潘思亮看好國內經濟發展，今年台灣經濟成長率可望名列亞洲四小龍之首。（圖／記者湯興漢攝）
記者林淑慧／台北報導
2020總統大選小英狂勝，飯店業龍頭、晶華酒店集團董事長潘思亮表示，外界普遍認為香港「反送中」為執政黨勝選利基，但他認為，蔡政府在2018年後轉以「拚經濟」政策為主軸，在中美貿易戰升溫成為科技戰的框架下，台商逐漸轉移陣地，成為三十年來回流台灣的最大一波浪潮，成為取得連任關鍵，並看好台灣前景將開啟黃金十年。
潘思亮表示，在中美貿易戰的大國角力下，過去單一的全球供應鏈成為兩套供應鏈，台商在遷移生產及研發版圖的同時，亦可望轉型升級，這是他回台三十年來首次見到台商回流的最大浪潮。
「政府的想法改變了，重要的是如何做，如果政府可以持續全力拚經濟為主的政策，帶動國內投資、消費及內需表現，台灣今年的經濟成長率將優於南韓、香港與新加坡，位居亞洲四小龍之首。」他說。
▲潘思亮指出人力與人才的不足，是台灣能否提升國際競爭力的關鍵。（圖／記者湯興漢攝）
「不過，台灣面臨最大的國安危機是人口老化、人口減少，」潘思亮也發出示警，指出未來最大的經濟危機因素將來自於企業「找不到人」而被迫關門，這樣的現象已在日本上演，台灣也很有可能步上日本後塵，建議政府必須鬆綁人口、勞工政策，和日本、香港、新加坡看齊。
針對產業界的五缺問題，潘思亮指出人力與人才的不足，是台灣能否提升國際競爭力的關鍵，為解決嚴重的缺工問題，日本政府制定國家級的戰略目標，開放外籍勞工投入觀光、建築、居家看護和農業等產業，目前已看出成效，值得台灣效法。
另外，台北晶華酒店今年適逢三十週年，晶華酒店董事總經理吳偉正表示，內部已陸續啟動改裝更新工程，除了先前改裝完畢的晶華軒外，牛排館與鐵板燒餐廳Robin’s的內外設備，將於下週改裝完畢，麗晶精品B1樓層也將於今年第一季分區改裝，屆時全館翻新完工，挹注業績表現可期。
依據財報，晶華2019年第三季稅後純益2.8億元，年增23%，每股稅後純益（EPS）2.19元，創七年來同期新高；累計2019年前三季稅後純益10.19億元，年增約0.5%，每股稅後純益（EPS）為8.03 元。
----------------------------------------------------------------------

[67] 台積電預告本季是最旺淡季　惟外資壓

▲仁寶副董事長陳瑞聰（左起）今日帶夫人參加仁寶尾牙。（圖／記者姚惠茹攝）
記者周康玉／台北報導
儘管日前鬧出和美容師小三赴日本泡湯的誹聞鬧得沸沸揚揚，仁寶副董事長陳瑞聰今（16）日不畏懼腥聞纏身，勇敢出席仁寶尾牙，更牽著夫人李敏芳敬酒同框。
陳瑞聰過去以愛家、怕老婆出名，儘管工作繁忙，但假日時間一定是老婆的，還自嘲自己越老越怕老婆。然而日前卻被周刊拍到，帶著小他27歲的美容師前往日本度假，全程5天4夜被跟拍。
然而尷尬的是，不僅與美容師旅日被拍到，連回國後，夫人親自接機畫面也被捕捉到，令人不勝唏噓。原本媒體預料陳瑞聰今日可能為了避嫌，不會出席尾牙活動，沒想到，陳瑞聰不僅照樣現身，還一路牽著李敏芳的手走紅毯。
陳瑞聰不僅大方帶老婆走紅毯，兩人還一起與同仁敬酒，有不少同事來要求合照，李敏芳不僅有求必應，還始終保持燦爛的露齒笑，和員工互動也超親切，不只點點頭小酌而已，只要有同事來敬酒，夫人都是酒杯高舉乾杯。 
----------------------------------------------------------------------

[80] 中美貿易戰紛爭仍在　南亞董座吳嘉昭提4大方向因應　

▲南亞董事長吳嘉昭。（圖／記者余弦妙攝）
記者余弦妙／台北報導
台塑集團旗下南亞公司今（16）日舉辦尾牙，會中南亞董事長吳嘉昭指出，去年化工整年都不好，尤其是中國大陸的擴充速度相當快，再加上中美貿易戰紛爭未平等因素，想讓化工「翻身」他也提出4大方向因應。
南亞去年因為主要各產品市場供給大多增加或持平，而因競爭激烈、產品價格偏低，部份產線不敷成本而停車減產，導致營收減少。其中以化工產品(EG及BPA為主)減少最多，其次為聚酯產品。全年稅後純益230.69億元，年減56.2%，衰退幅度是四寶最大，每股純益2.91元。
吳嘉昭說，去年是相當不好的一年，中美貿易戰影響全球市場信心，而展望今年，全球仍受到在貿易戰、科技戰、匯率戰與地緣政治的影響，而第1季受到春節假期影響，1月營收會較去年12月衰退，產品價格見不到上漲空間，第1季也會比第4季衰退。
不過，吳嘉昭也提出4大方向因應，首先，他認為要先擴充市場，讓產能增加，去年化工產品雖然停產、減產，但今年會逐漸復工，因此今年一定要超過去年的水準。
再來則是研究開發，朝高質化、差異化的方向努力，特別是南亞是加工業，研發就更為重要

▲東森購物、日本宮崎電視於16日簽訂完成合作備忘錄簽署。左起為東森購物執行長彭鴻珷、宮崎電視社長津隈佳三、TMJ社長高田智康。（圖／東森購物提供，以下同）
記者洪菱鞠／台北報導
東森購物推進國際，16日宣布與日本宮崎電視台簽訂合約書，未來雙方將積極在商品引進及節目製播技術上展開全面性交流；東森購物執行長彭鴻珷認為，對於東森購物、宮崎電視、台灣及日本的消費者來說，可望創造4贏局面，而預計第二季開始，將有宮崎深度旅遊、相關農產品等陸續在東森獨家推出。
彭鴻珷指出，日本向來是台灣人最愛去的地方，尤其今年到日本旅遊預計將超過600萬人次，其顯示出對於日本的認同和喜愛，東森也落實「東森嚴選」精神，不僅持續參加每年3月的食品產，近年也精選販售來自日本的芝麻醬、湯淺醬油、水蜜桃、青森蘋果等好物，讓消費者出國不用再大包小包帶回家。
對於此次與宮崎電視簽署合作，彭鴻珷提到，宮崎縣擁有全日本最優質的牛肉品種、稻米產地以及豐富旅遊景點，雙方在簽訂合約書後，樂見未來能夠把當地最好的產品直接帶給台灣消費者，其中，今年3～6月東森購物將獨家引進日本MRT宮崎電視所精心挑選在地的品牌，另一方面，宮崎電視也將同步銷售東森自然美近年暢銷的「奇肌無瑕活膚系列」以及「瑞昇超逆齡賦活膠囊」，把台灣的好商品輸入到日本去。
▲未來雙方將積極在商品引進及節目製播技術上展開全面性交流。
宮崎電視是當地市占率第一大的媒體，本身是電視台之外，也有廣播，另外還跨足貿易事業，於2018年成立子公司「株式會社Trade Media Japan」，作為進行台日文化交流的角色。而東森購物是宮崎電視在海外合作的第一家電視台，社長津隈佳三說，東森購物在台灣擁有最多頻道，同時在東南亞也有很多銷售網絡，相信雙方合作可以創造更大商機，並且把日本名產介紹到各地。
津隈佳三指出，宮崎被稱為日本的大糧倉，所生產的米、肉類、魚、蔬菜、水果等，不論哪樣都很美味，尤其很多當地特產在日本已經是很高級，海外卻沒有太多人知道，希望透過與東森購物長期合作，能夠把這些物產尤其是農產品推廣出去。
TMJ社長高田智康則說，宮崎電視與東森購物的合作，將透過各自的播放媒體及商品，促進台日社會文化交流與經濟發展，雙方免費提供播放素材，並在協商後共同從事採訪活動和節目製作，銷售彼此的商品。此次特別選擇東森自然美兩項產品，主要在於東森自然美是亞洲相當具有影響力的專業美容S